# Marketing Analysis -- Final Project 
## Code Session 
## Yiyin Zhang (yz7067)

# 0. Visualization

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

fig, ax = plt.subplots(1, 1, figsize=(15, 10))

plt.tick_params(labelsize=15)

a = np.array(cust['age'])
ax.hist(a, bins = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,105,110], rwidth=0.5)
#ax.set_title('age', fontsize=20)
ax.set_xticks([0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,105,110])
ax.set_xlabel('age', fontsize=20)
ax.set_ylabel('subscriber count', fontsize=20)

plt.savefig('age distribution.png')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=2)
f, ax = plt.subplots(figsize=(45,10))
sns.countplot(x='num_ideal_streaming_services',data=df)#palette = ['#8e1f0b','#db6d25ff', '#ffb03bff','#ffce4bff'])
#pd.set_option('display.float_format', lambda x: '%.10f' % x)
plt.ticklabel_format(style='plain', axis='y')

# 1. Customer Segmentation

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('subscribers-updated.csv')
df = df.drop(['Unnamed: 0'], axis=1,)
df.info()

In [ ]:
for n in range(0,len(df)):
    if df.loc[n,'age'] > 2018:
        df.loc[n,'age'] = None
    elif df.loc[n,'age'] > 1929:
        df.loc[n,'age'] = 2019 - df.loc[n,'age']
    elif df.loc[n,'age'] > 108:
        df.loc[n,'age'] = None

In [ ]:
#Drop age's negative value
df=df[df['age']>0]
#Drop weekly_consumption_hour's negative value
df=df[df['weekly_consumption_hour']>0]
#Drop num_ideal_streaming_services's negative value
df=df[df['num_ideal_streaming_services']>0]

In [ ]:
df.dropna(subset=['package_type','preferred_genre','intended_use','weekly_consumption_hour','age','op_sys','male_TF'],inplace=True)
df.info()

In [ ]:
cust = df[['subid', 'package_type', 'num_weekly_services_utilized',
       'preferred_genre', 'intended_use', 'weekly_consumption_hour','num_ideal_streaming_services',
       'age', 'male_TF','attribution_technical', 'op_sys']]
cust.info()

In [ ]:
numweek = cust.num_weekly_services_utilized.mean()
numideal = cust.num_ideal_streaming_services.mean()
cust['num_weekly_services_utilized'].fillna(numweek,inplace=True)
cust['num_ideal_streaming_services'].fillna(numideal,inplace=True)

In [ ]:
custdum=pd.get_dummies(cust,drop_first = True)
custdum.info()
custdumnoid = custdum.iloc[:,1:50] #Remove subid columns
custdumnoid.info()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmaxscaler = MinMaxScaler()
custdumnoid_scaled = minmaxscaler.fit_transform(custdumnoid) 

## inertia 

In [ ]:
from sklearn.cluster import KMeans
# criterion: inertia #看平的点
inertias = {}
for k in range(2, 20):
    kmeans = KMeans(n_clusters=k, random_state=2020)
    kmeans.fit(custdumnoid_scaled)  # here we do not use fit_transform since we don't need the transformed result
    inertias[k] = kmeans.inertia_
print(inertias) 

In [ ]:
import matplotlib.pyplot as plt #select: 5?
f, ax = plt.subplots(figsize=(30,10))
ax = plt.subplot()
ax.plot(list(inertias.keys()), list(inertias.values()) ,'-*')
ax.set_xticks(np.arange(2, 20))
#ax.grid()
plt.show()

In [ ]:
from sklearn.cluster import KMeans
k = 4
kmeans = KMeans(n_clusters=k, random_state=1996)
y_pred = kmeans.fit_predict(custdumnoid_scaled)
y_pred

In [ ]:
#put the cluster number back to 
custclus = pd.DataFrame(cust)
custclus['cluster'] = y_pred
custclus['cluster'].value_counts()
custclus=custclus.reset_index()
custclus.drop('index',axis='columns', inplace=True)

In [ ]:
custclus.to_csv('cluster4F.csv')

# 2. Attribution Allocation

In [ ]:
import pandas as pd
import time
from datetime import datetime as dt

In [ ]:
## Considering the user converted with revenue generated: 
## and considering spend on that channel worths 
### Payment_period >= 1

In [ ]:
df = pd.read_csv('subscribers-updated.csv',parse_dates=['account_creation_date'])
df = df.drop(['Unnamed: 0'], axis=1)
spe = pd.read_csv('channel_spend_graduate.csv')
spe1 = pd.read_csv('channel_spend_graduate.csv')

In [ ]:
att = df[['subid', 'attribution_technical','attribution_survey','account_creation_date','payment_period','trial_completed']]
att.dropna(subset = ['attribution_survey'],inplace = True)

In [ ]:
#filtering out paymeent_period == 0 and trial_completed = True
att1 = att[(att['payment_period']!=0) & (att['trial_completed'] == True)]
att1.reset_index(inplace=True)
att1.drop('index',axis='columns', inplace=True)

att1.loc[:,('date')] = att1['account_creation_date'].dt.strftime('%Y%m')
att1.loc[:,('date')] = att1['date'].astype('int')

### Attribution_Technical 

In [ ]:
for m in range(0,len(spe)):
    channel = spe.loc[m,'channel']
    date = spe.loc[m,'date']
    subattri= att1[(att1['date'] == date) & (att1['attribution_technical'] == channel)]
    spe.loc[m,'count'] = len(subattri)
    spe.loc[m,'Average_CAC_Tech'] = spe.loc[m,'spend_AED']/spe.loc[m,'count']

In [ ]:
speMt = spe.sort_values(['channel','spend_AED'],ascending=True)
speMt.reset_index(inplace=True)
speMt.drop('index',axis='columns', inplace=True)

In [ ]:
for n in range(1,len(speMt)):
    if speMt.loc[n,'channel'] == speMt.loc[(n-1),'channel']:
        speMt.loc[n,'Marginal_CAC_Tech'] = (speMt.loc[n,'spend_AED'] - speMt.loc[(n-1),'spend_AED'])/(speMt.loc[n,'count'] - speMt.loc[(n-1),'count'])
    else:
        continue

for m in [0,8,16,24,32]:
    speMt.loc[m,'Marginal_CAC_Tech'] = speMt.loc[m,'spend_AED'] /speMt.loc[m,'count']

In [ ]:
speMt.to_csv("Attri_Technical.csv")

### Attribution_Survey

In [ ]:
for m in range(0,len(spe1)):
    channel = spe1.loc[m,'channel']
    date = spe1.loc[m,'date']
    subattri= att1[(att1['date'] == date) & (att1['attribution_survey'] == channel)]
    spe1.loc[m,'count'] = len(subattri)
    spe1.loc[m,'Average_CAC_Sur'] = spe1.loc[m,'spend_AED']/spe1.loc[m,'count']

In [ ]:
speMs = spe1.sort_values(['channel','spend_AED'],ascending=True)
speMs.reset_index(inplace=True)
speMs.drop('index',axis='columns', inplace=True)
speMs = speMs.loc[16:39,]
speMs.reset_index(inplace=True)
speMs.drop('index',axis='columns', inplace=True)

In [ ]:
for n in range(1,len(speMs)):
    if speMs.loc[n,'channel'] == speMs.loc[(n-1),'channel']:
        speMs.loc[n,'Marginal_CAC_Sur'] = (speMs.loc[n,'spend_AED'] - speMs.loc[(n-1),'spend_AED'])/(speMs.loc[n,'count'] - speMs.loc[(n-1),'count'])
    else:
        continue
for m in [0,8,16]:
    speMs.loc[m,'Marginal_CAC_Sur'] = speMs.loc[m,'spend_AED'] /speMs.loc[m,'count']

In [ ]:
speMs.to_csv("Attri_Survey.csv")

# 3. Churn Model

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import seaborn as sn
import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [ ]:
df = pd.read_csv('subscribers-updated.csv')
df = df.drop(['Unnamed: 0'], axis=1,)
df

In [ ]:
# Dealing with abnormal value
for n in range(0,len(df)):
    if df.loc[n,'age'] > 2018:
        df.loc[n,'age'] = None
    elif df.loc[n,'age'] > 1929:
        df.loc[n,'age'] = 2019 - df.loc[n,'age']
    elif df.loc[n,'age'] > 108:
        df.loc[n,'age'] = None

# drop negative value:
df=df[df['age']>0]
df=df[df['weekly_consumption_hour']>0]
df=df[df['num_ideal_streaming_services']>0]

numweek = df.num_weekly_services_utilized.mean()
numideal = df.num_ideal_streaming_services.mean()
df['num_weekly_services_utilized'].fillna(numweek,inplace=True)
df['num_ideal_streaming_services'].fillna(numideal,inplace=True)

## Create x-value form 

In [ ]:
churn = df[['subid', 'package_type', 'num_weekly_services_utilized',
       'preferred_genre', 'intended_use', 'weekly_consumption_hour',
        'num_ideal_streaming_services','age', 'male_TF','attribution_technical',
        'op_sys','join_fee','plan_type','payment_period','account_creation_date','cancel_date']]

churn['account_creation_date'] = pd.to_datetime(churn['account_creation_date'])

In [ ]:
churn =  churn[churn['plan_type']=='base_uae_14_day_trial']
churn.dropna(subset=['package_type','preferred_genre'],inplace=True)
churn = churn.reset_index()
churn.drop('index',axis='columns', inplace=True)

### 3.1. Create user churn Y = 1, p = 0

##### 1.1 period_0 + Trial End

In [ ]:
trialend1 = churnp0[((churnp0['account_creation_date']<= '2020-03-13 00:00:00') == True)]
trialend2 = churnp0[(churnp0['cancel_date'].isnull() == True)]
trialend = churnp0[((churnp0['account_creation_date']<= '2020-03-13 00:00:00') == True) & (churnp0['cancel_date'].isnull() == True)]
len(trialend)

##### 1.2 period_0 + Active Cancel

In [ ]:
actcancel = churnp0[(churnp0['cancel_date'].isnull() == False)]
len(actcancel)

In [ ]:
# Combine 
combine = [trialend,actcancel]
churny1 = pd.concat(combine)
len(churn)
churny1['churn'] = 1
churny1

### 3.2. Create user churn Y = 0,  p=1

In [ ]:
#nochurn = churn[((churn['payment_period'] > 0) == True)]
churny0 = churn[churn['payment_period'] > 0]
churny0['churn'] = 0
len(churny0)

### 3.3. Combine Y =1 & Y= 0

In [ ]:
combine = [churny1,churny0]
trval =  pd.concat(combine)

trval = trval.reset_index()
trval.drop('index',axis='columns', inplace=True)
#trval

### 4. user-not decided Y = ?, p=0

In [ ]:
undecide1 = churnp0[((churnp0['account_creation_date']<= '2020-03-13 00:00:00') == False)]
undecide2 = churnp0[(churnp0['cancel_date'].isnull() == True)]
undecide = churnp0[((churnp0['account_creation_date']<= '2020-03-13 00:00:00') == False) & (churnp0['cancel_date'].isnull() == True)]
undecide

## Pick x-features & fit model

In [ ]:
Xfinal = trval[['package_type', 'num_weekly_services_utilized',
       'preferred_genre', 'intended_use', 'weekly_consumption_hour',
       'num_ideal_streaming_services', 'age', 'male_TF',
       'attribution_technical', 'op_sys', 'join_fee']]
yfinal = trval[['churn']]

#X_un = undecide[['package_type', 'num_weekly_services_utilized',
#       'preferred_genre', 'intended_use', 'weekly_consumption_hour',
#       'num_ideal_streaming_services', 'age', 'male_TF',
#       'attribution_technical', 'op_sys', 'join_fee']]

In [ ]:
Xfinal =  pd.get_dummies(Xfinal,drop_first = True)
#X_un =  pd.get_dummies(X_un,drop_first = True)
#X_un.info()
#yfinal.info()

In [ ]:
#seperate train-test sets
X_train, X_val, y_train, y_val = train_test_split(Xfinal, yfinal, test_size=0.30, random_state=1997)
X_train.info()

In [ ]:
#normalization with minmax
#change from dataframe to array
minmaxscaler = MinMaxScaler()
X_train = minmaxscaler.fit_transform(X_train) 
X_val = minmaxscaler.fit_transform(X_val) 
#X_un = minmaxscaler.fit_transform(X_un) 

In [ ]:
# fit a model
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

y_pred=logreg.predict(X_val)

# predict probabilities
y_pred_prob=logreg.predict_proba(X_val)

# keep probabilities for the positive outcome only
y_pred_prob = y_pred_prob[:, 1]
# from 2-dimension to 1-dimension 
y_valTranf= y_val['churn'].values

In [ ]:
# auc score
log_auc = roc_auc_score(y_valTranf, y_pred_prob)
log_auc

In [ ]:
confusion_matrix = pd.crosstab(y_valTranf,y_pred,rownames=['Actual'], colnames=['Predicted'])
sn.heatmap(confusion_matrix, annot=True)
confusion_matrix

In [ ]:
print('Accuracy: ',metrics.accuracy_score(y_val, y_pred))

print('Recall: ',metrics.recall_score(y_val, y_pred))

print('Precision: ',metrics.precision_score(y_val, y_pred))
